In [11]:
from langchain_community.tools import tool
import requests

In [12]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [13]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [14]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'BDT'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1761523201,
 'time_last_update_utc': 'Mon, 27 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1761609601,
 'time_next_update_utc': 'Tue, 28 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 122.3402}

In [16]:
convert.invoke({'base_currency_value':10, 'conversion_rate':122.3402})

1223.402

## Tool binding

In [17]:
from langchain_groq.chat_models import ChatGroq


llm=ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [18]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [21]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage('What is the conversion factor between BDT and USD, and based on that can you convert 10000 bdt to usd')]

In [22]:
messages

[HumanMessage(content='What is the conversion factor between BDT and USD, and based on that can you convert 10000 bdt to usd', additional_kwargs={}, response_metadata={})]

In [23]:
ai_message = llm_with_tools.invoke(messages)

In [24]:
messages.append(ai_message)

In [25]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'BDT', 'target_currency': 'USD'},
  'id': 'k7ws50vgd',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10000},
  'id': 'cxsdk7npn',
  'type': 'tool_call'}]

In [26]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [27]:
messages

[HumanMessage(content='What is the conversion factor between BDT and USD, and based on that can you convert 10000 bdt to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'k7ws50vgd', 'function': {'arguments': '{"base_currency":"BDT","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'cxsdk7npn', 'function': {'arguments': '{"base_currency_value":10000}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 351, 'total_tokens': 390, 'completion_time': 0.08344462, 'prompt_time': 0.032912781, 'queue_time': 0.048374308, 'total_time': 0.116357401}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--cb9c4354-4fc0-4353-a9c7-c890c217f3df-0', tool_calls=[{'name': 'get_conversion_facto

In [28]:
llm_with_tools.invoke(messages).content

'The conversion factor between BDT and USD is 0.008174. Based on this conversion factor, 10000 BDT is equivalent to approximately 81.74 USD.'